In [66]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Your Ticketmaster API key
apikey = "XgRNVlqcG3ANgyw6dE9BgWVdX7jy4lb0"

# Base URL for the API
base_url = "https://app.ticketmaster.com/discovery/v2/events.json"

# Initial parameters
params = {
    "apikey": apikey,
    "countryCode": "US",
    "size": 200,  # Maximum size per request
    "sort": "date,asc",
    "page": 0,   # Start from the first page
}

# Container for all events
all_events = []
start_date = datetime.now()  # Start from today
current_page = 0
page_max = 20
previous_start_date = None  # To prevent infinite loops caused by redundant date updates

In [67]:
while True:
    # Error handling for infinite loops caused by redundant dates
    if start_date == previous_start_date:
        print("Detected redundant date update. Increasing date to prevent infinite requests.")
        last_event_date = datetime.strptime(all_events[-1]["start_date"], "%Y-%m-%d")
        start_date = last_event_date + timedelta(days=2)

    previous_start_date = start_date  # Update the previous start date
    params["startDateTime"] = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    params["page"] = 0  # Reset to the first page for this start date

    print(f"Fetching events starting from {params['startDateTime']}...")

    for page in range(5):  # Fetch up to 5 pages for the current date
        params["page"] = page

        # Make the API request
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()  # Raise an HTTPError for bad responses
            data = response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            break

        # Extract events
        events = data.get("_embedded", {}).get("events", [])
        if not events:  # No more events found
            print("No more events available.")
            break

        # Process each event
        for event in events:
            try:
                # Extract event details
                event_info = {
                    "event_id": event.get("id"),
                    "event_name": event.get("name", None),
                    "event_type": event.get("type", None),
                    "event_url": event.get("url", None),
                    "locale": event.get("locale", None),
                    "start_date": event.get("dates", {}).get("start", {}).get("localDate", None),
                    "start_time": event.get("dates", {}).get("start", {}).get("localTime", None),
                    "timezone": event.get("dates", {}).get("timezone", None),
                    "status": event.get("dates", {}).get("status", {}).get("code", None),
                }

                # Extract venue information
                venues = event.get("_embedded", {}).get("venues", [{}])
                venue_info = {
                    "venue_id": venues[0].get("id", None),
                    "venue_name": venues[0].get("name", None),
                    "venue_city": venues[0].get("city", {}).get("name", None),
                    "venue_state": venues[0].get("state", {}).get("name", None),
                    "venue_country": venues[0].get("country", {}).get("name", None),
                    "venue_address": venues[0].get("address", {}).get("line1", None),
                    "venue_postal_code": venues[0].get("postalCode", None),
                    "venue_latitude": venues[0].get("location", {}).get("latitude", None),
                    "venue_longitude": venues[0].get("location", {}).get("longitude", None),
                }

                # Extract classification (genre) information
                classifications = event.get("classifications", [{}])
                classification_info = {
                    "segment_name": classifications[0].get("segment", {}).get("name", None),
                    "genre_name": classifications[0].get("genre", {}).get("name", None),
                    "sub_genre_name": classifications[0].get("subGenre", {}).get("name", None),
                }

                # Extract attraction information
                attractions = event.get("_embedded", {}).get("attractions", [{}])
                attraction_info = {
                    "attraction_id": attractions[0].get("id", None),
                    "attraction_name": attractions[0].get("name", None),
                }

                # Extract price range information
                price_range = event.get("priceRanges", [{}])
                price_info = {
                    "min_price": price_range[0].get("min", None),
                    "max_price": price_range[0].get("max", None),
                    "currency": price_range[0].get("currency", None),
                }

                # Combine all extracted information
                combined_info = {**event_info, **venue_info, **classification_info, **attraction_info, **price_info}
                all_events.append(combined_info)

            except Exception as e:
                print(f"Error processing event: {e}")
                continue

        # Check if the current page is the last page
        if data.get("page", {}).get("totalPages", 1) - 1 == page:
            print("Reached the last page for this start date.")
            break

    # Update start_date to the last event's date + 1 day to avoid overlaps
    if all_events:
        last_event_date = datetime.strptime(all_events[-1]["start_date"], "%Y-%m-%d")
        start_date = last_event_date + timedelta(days=1)
    else:
        print("No events found to update the start date.")
        break


Fetching events starting from 2024-12-02T17:14:22Z...
Fetching events starting from 2024-12-04T00:00:00Z...
Fetching events starting from 2024-12-05T00:00:00Z...
Fetching events starting from 2024-12-06T00:00:00Z...
Detected redundant date update. Increasing date to prevent infinite requests.
Fetching events starting from 2024-12-07T00:00:00Z...
Detected redundant date update. Increasing date to prevent infinite requests.
Fetching events starting from 2024-12-08T00:00:00Z...
Detected redundant date update. Increasing date to prevent infinite requests.
Fetching events starting from 2024-12-09T00:00:00Z...
Fetching events starting from 2024-12-10T00:00:00Z...
Fetching events starting from 2024-12-11T00:00:00Z...
Fetching events starting from 2024-12-12T00:00:00Z...
Fetching events starting from 2024-12-13T00:00:00Z...
Fetching events starting from 2024-12-14T00:00:00Z...
Detected redundant date update. Increasing date to prevent infinite requests.
Fetching events starting from 2024-12-15

KeyboardInterrupt: 

In [68]:
all_events

[{'event_id': '1AeFZbaGkB3eo2V',
  'event_name': 'Mosconi Cup 2024',
  'event_type': 'event',
  'event_url': 'https://www.ticketmaster.com/mosconi-cup-2024-orlando-florida-12-02-2024/event/2200609E920913F4',
  'locale': 'en-us',
  'start_date': '2024-12-02',
  'start_time': '13:00:00',
  'timezone': 'America/New_York',
  'status': 'onsale',
  'venue_id': 'KovZpZA7eaIA',
  'venue_name': 'Caribe Royale Orlando',
  'venue_city': 'Orlando',
  'venue_state': 'Florida',
  'venue_country': 'United States Of America',
  'venue_address': '8101 World Center Drive',
  'venue_postal_code': '32821',
  'venue_latitude': '28.35749200',
  'venue_longitude': '-81.49340600',
  'segment_name': 'Sports',
  'genre_name': 'Miscellaneous',
  'sub_genre_name': 'Miscellaneous',
  'attraction_id': 'K8vZ917oO5f',
  'attraction_name': 'Mosconi Cup',
  'min_price': 70.0,
  'max_price': 500.0,
  'currency': 'USD'},
 {'event_id': 'Za5ju3rKuqZB0JrpXv_We8znm28cgibgP',
  'event_name': 'Shark Reef Aquarium',
  'event_ty

In [69]:
# Save the data to a CSV file
if all_events:
    df = pd.DataFrame(all_events)
    df.to_csv("events_usa.csv", index=False)
    print("Data saved to 'events_usa.csv'.")
else:
    print("No data collected.")

Data saved to 'events_usa.csv'.


In [72]:
df.shape

(135230, 26)